<center><h1>第四章 分组</h1></center>

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('../data/learn_pandas.csv')
df.head()

,School,Grade,Name,Gender,Height,Weight,Transfer,Test_Number,Test_Time,Time_Record
0,Shanghai Jiao Tong University,Freshman,Gaopeng Yang,Female,158.9,46.0,N,1,2019/10/5,0:04:34
1,Peking University,Freshman,Changqiang You,Male,166.5,70.0,N,1,2019/9/4,0:04:20
2,Shanghai Jiao Tong University,Senior,Mei Sun,Male,188.9,89.0,N,2,2019/9/12,0:05:22
3,Fudan University,Sophomore,Xiaojuan Sun,Female,NaN,41.0,N,2,2020/1/3,0:04:08
4,Fudan University,Sophomore,Gaojuan You,Male,174.0,74.0,N,2,2019/11/6,0:05:22


In [10]:
pd.DataFrame({"Weight":[60,70,80],"Height":[176,180,179]}, index=['1001','1003','1002']).reindex(index=['1001','1002','1003','1004'],columns=['Weight','Gender'])

,Weight,Gender
1001,60.0,NaN
1002,80.0,NaN
1003,70.0,NaN
1004,NaN,NaN


In [17]:
my_list1 = ['a','b']
my_list2 = ['cat','dog']
pd.MultiIndex.from_product([my_list1,
                            my_list2],
                           names=['First','Second'])

MultiIndex([('a', 'cat'),
            ('a', 'dog'),
            ('b', 'cat'),
            ('b', 'dog')],
           names=['First', 'Second'])